In [1]:
#Before using spark, need to install and set up spark, Hadoop and Jdk locally on your laptop


from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("etl") \
    .config("spark.master", "local") \
    .config("spark.jars", "F:/UniPrj/Big Data Project/postgresql-42.2.16.jar") \
    .getOrCreate()

#Load Amazon Data into Spark DataFrame
#1 load product segment - Office Products Segment --> download dataset tsv file
file_path = "F:/UniPrj/Big Data Project/Datasets/amazon_reviews_us_Office_Products_v1_00.tsv"
df = spark.read.option("encoding", "UTF-8").csv(file_path, sep="\t", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [2]:
#Create DataFrame - Perform Preliminary Cleaning

#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)

(2642434, 15)


In [3]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|   43081963|R18RVCKGH1SSI9|B001BM2MAC|Scotch Cushion Wr...| Office Products|          5|            0|          0|                Y|
|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|Dust-Off Compress...| Office Products|          5|            0|          1|                Y|
|   21143145|R2J8AWXWTDX2TF|B00RTMUHDW|Amram Tagger Stan...| Office Products|          5|            0|          0|                Y|
|   52782374|R1PR37BR7G3M6A|B00D7H8XB6|AmazonBasics 12-S...| Office Products|          1|            2|          3|                Y|
|   24045652|R3BDDDZMZBZDPU|B001XCWP34|Derwent Colored P...| O

In [4]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))
     

(2257866, 9)


In [5]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|   43081963|R18RVCKGH1SSI9|B001BM2MAC|Scotch Cushion Wr...| Office Products|          5|            0|          0|
|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|Dust-Off Compress...| Office Products|          5|            0|          1|
|   21143145|R2J8AWXWTDX2TF|B00RTMUHDW|Amram Tagger Stan...| Office Products|          5|            0|          0|
|   52782374|R1PR37BR7G3M6A|B00D7H8XB6|AmazonBasics 12-S...| Office Products|          1|            2|          3|
|   24045652|R3BDDDZMZBZDPU|B001XCWP34|Derwent Colored P...| Office Products|          4|            0|          0|
|   21751234| R8T6MO75ND212|B004J2NBCO|Quartet Magnetic ...| Office Prod

In [6]:
# Create Analysis-Specific DFs/Tables
# Perform Analysis-Specific Transforms
# Segmentation Analysis DF

# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   43081963| Office Products|
|   10951564| Office Products|
|   21143145| Office Products|
|   52782374| Office Products|
|   24045652| Office Products|
|   21751234| Office Products|
|    9967215| Office Products|
|   11234247| Office Products|
|   12731488| Office Products|
|    1541556| Office Products|
|   10579374| Office Products|
|   43622307| Office Products|
|   50773856| Office Products|
|   10165223| Office Products|
|   14744748| Office Products|
|   37536631| Office Products|
|     126687| Office Products|
|   31130095| Office Products|
|   14300387| Office Products|
|   22083223| Office Products|
+-----------+----------------+
only showing top 20 rows



In [7]:
#Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')


#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'office_products')
#2c Check results
segment_df.show()
     

+-----------+---------------+
|customer_id|office_products|
+-----------+---------------+
|   43622307|              1|
|   33972406|              1|
|     108460|              1|
|   43515569|              1|
|     132406|              1|
|   42560427|              1|
|    1673863|              2|
|   14552054|              1|
|   45632184|              1|
|   14703850|              2|
|   35768925|              3|
|   45640757|             28|
|   33194403|              1|
|   39383689|              1|
|   10645124|              1|
|   52512151|             10|
|   14646714|              1|
|   38659596|              1|
|   15634680|              1|
|   44178035|              1|
+-----------+---------------+
only showing top 20 rows



In [8]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- office_products: long (nullable = false)

1557076


In [9]:
#Note: Column Name in df.sort('name' ...) must align with Column name from step 2


#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('office_products', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+---------------+
|customer_id|office_products|
+-----------+---------------+
|   46054413|            121|
|    5044927|            117|
|   16016941|            104|
|   50749068|            101|
|   44115969|             99|
|   44062423|             99|
|   39061233|             97|
|   51148531|             84|
|   35321242|             79|
|   50820654|             76|
|   11474525|             72|
|   13521389|             69|
|     544143|             69|
|   38398088|             69|
|   52443411|             69|
|   51126130|             68|
|   11451031|             66|
|   49452274|             66|
|   33928846|             66|
|   41309704|             66|
+-----------+---------------+
only showing top 20 rows

100000


In [10]:
#Segmentation ETL Complete - Add Database Export Code

# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}

In [11]:
#Note: table name in table='name_segment' must align with table name in Postgres


# Write segment table to Postgres/RDS
# xx mins
filtered_segment_df.write.jdbc(url=jdbc_url, table='office_products_segment', mode=mode, properties=config)
     

In [12]:
# Run Queries in Postgres to Confirm Segment ETL
# Check Row Count of Segment Table - SELECT COUNT(*) FROM office_products_segment;

# Check 10 Rows of Segment Table - SELECT * FROM office_products_segment LIMIT(10);


# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful
